# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [6]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [8]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [11]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier.

In [12]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..principal creation[##################################]  100.0000%
  "additionalProperties": {},
  "agentPoolProfiles": [
    {
      "additionalProperties": {},
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-1d0981c7.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSCluster",
  "kubernetesVersion": "1.7.9",
  "linuxProfile": {
    "additionalProperties": {},
    "adminUsername": "azureuser",
    "ssh": {
      "additionalProperties": {},
      "publicKeys": [
        {
          "additionalProperties": {},
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADA

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [12]:
!sudo az aks install-cli

## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [13]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSCluster" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [14]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     11m       v1.7.9


Let's check the pods on our cluster.

In [15]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-2574232661-dz21t               2/2       Running   0          12m
kube-system   kube-dns-v20-2253765213-6z48m           3/3       Running   0          14m
kube-system   kube-dns-v20-2253765213-htqfl           3/3       Running   0          14m
kube-system   kube-proxy-50n4d                        1/1       Running   0          13m
kube-system   kube-svc-redirect-lz97p                 1/1       Running   0          13m
kube-system   kubernetes-dashboard-2898242510-g0rlq   1/1       Running   0          14m
kube-system   tunnelfront-3141404306-34mj0            1/1       Running   0          14m


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [16]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerastf-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "name": "bin",
                              "mountPath":"/usr/local/nvidia/bin" 
                          },
                          {
                              "name": "lib",
                              "mountPath":"/usr/local/nvidia/lib64" 
                          },
                          {
                              "name": "libcuda",
                              "mountPath":"/usr/lib/x86_64-linux-gnu/libcuda.so.1" 
                          },
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "bin",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384/bin"
                      },
                  },
                  {
                      "name": "lib",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384"
                      },
                  },
                  {
                      "name": "libcuda",
                      "hostPath":{
                          "path":"/usr/lib/x86_64-linux-gnu/libcuda.so.1"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [17]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [18]:
write_json_to_file(app_template, 'az-dl.json')

In [19]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [20]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "fboylu/kerastf-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
               

Next, we will use kubectl create command to deploy our application.

In [21]:
!kubectl create -f az-dl.json

deployment "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [22]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS              RESTARTS   AGE
default       azure-dl-1885859937-8vrkv               0/1       ContainerCreating   0          40s
kube-system   heapster-2574232661-dz21t               2/2       Running             0          17m
kube-system   kube-dns-v20-2253765213-6z48m           3/3       Running             0          18m
kube-system   kube-dns-v20-2253765213-htqfl           3/3       Running             0          18m
kube-system   kube-proxy-50n4d                        1/1       Running             0          18m
kube-system   kube-svc-redirect-lz97p                 1/1       Running             0          18m
kube-system   kubernetes-dashboard-2898242510-g0rlq   1/1       Running             0          18m
kube-system   tunnelfront-3141404306-34mj0            1/1       Running             0          18m


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [23]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                             SOURCE                                 MESSAGE
19m         24m          7         aks-nodepool1-28016997-0.152241072750679b    Node                                     Normal    NodeHasSufficientDisk              kubelet, aks-nodepool1-28016997-0      Node aks-nodepool1-28016997-0 status is now: NodeHasSufficientDisk
19m         24m          7         aks-nodepool1-28016997-0.152241072750d04b    Node                                     Normal    NodeHasSufficientMemory            kubelet, aks-nodepool1-28016997-0      Node aks-nodepool1-28016997-0 status is now: NodeHasSufficientMemory
19m         24m          7         aks-nodepool1-28016997-0.152241072750f823    Node                                     Normal    NodeHasNoDiskPressure              kubelet, aks-nodepool1-28016997-0      Node aks-nodepool1-28016997-0 status

Check the logs for the application pod.

In [24]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [27]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-04-04 14:37:42,384 CRIT Supervisor running as root (no user in config file)
2018-04-04 14:37:42,386 INFO supervisord started with pid 8
2018-04-04 14:37:43,388 INFO spawned: 'program_exit' with pid 18
2018-04-04 14:37:43,389 INFO spawned: 'nginx' with pid 19
2018-04-04 14:37:43,391 INFO spawned: 'gunicorn' with pid 20
2018-04-04 14:37:44,435 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-04-04 14:37:44.880056: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-04-04 14:37:48,887 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-04-04 14:37:54.419320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: f542:00:00.0
totalMemory: 11.17GiB freeMe

In [28]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           28m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [29]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
azure-dl   LoadBalancer   10.0.107.160   52.224.106.118   80:31114/TCP   28m


# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [4]:
!kubectl delete -f az-dl.json

deployment "azure-dl" deleted
service "azure-dl" deleted


In [5]:
!az aks delete -n $aks_name -g $resource_group -y

Deployment failed. Correlation ID: 4535be5c-80cf-4c58-a2fa-7feef2577a42. Unexpected response trying to delete control plane


In [6]:
!az group delete --name $resource_group -y

inished ..

Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb).